In [52]:
import torch
import torchvision.transforms as T
import torch.nn as nn
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from einops import rearrange

Here we want to create a validation set from the train set. We want a 85% train and 15%, keeping the labels proportionality

In [ ]:
TRAIN_CSV = 'EPIC_100_train.csv'
VAL_CSV = 'EPIC_100_validation.csv'

### Original df

In [ ]:
df = pd.read_csv(TRAIN_CSV)
df.head(5)

In [ ]:
df.shape

In [ ]:
df['verb_class'].apply(lambda x: int(x))

In [ ]:
df = df.drop(df[df['video_id'] == 'P23_04'].index)

## Create splits and new train/val dfs

In [ ]:
verbs = df['verb_class'].unique()

In [ ]:
df_verb_train_list = []
df_verb_val_list = []
for verb in verbs:
    if len(df[df['verb_class'] == verb]) >= 2:
        df_verb = df[df['verb_class'] == verb]
        df_verb_train, df_verb_val = train_test_split(df_verb, test_size=0.15, train_size=0.85, random_state=1, shuffle=True)
        df_verb_train_list.append(df_verb_train)
        df_verb_val_list.append(df_verb_val)
    else:
        print(f'verb {verb} skipped. Only {len(df[df["verb_class"] == verb])} values')

In [ ]:
df_train = pd.concat(df_verb_train_list)
df_val = pd.concat(df_verb_val_list)

In [ ]:
df_train['verb_class'].value_counts()

We have to make the classes to be in order of [0, 1, 2, .., 96]. Now in the last labels we have [0, ..., 92, 94, 95, 96] which raises an error.

In [ ]:
for num in [94, 95, 96]:
    for index in df_train['verb_class'][df_train['verb_class'] == num].index:
        df_train.loc[index, 'verb_class'] = num-1

In [ ]:
for num in [94, 95, 96]:
    for index in df_val['verb_class'][df_val['verb_class'] == num].index:
        df_val.loc[index, 'verb_class'] = num-1

In [ ]:
df_train['verb_class'].unique()

In [ ]:
df_val['verb_class'].unique()

In [ ]:
# shuffle the rows
df_train = df_train.sample(frac = 1)
df_val = df_val.sample(frac = 1)

In [ ]:
print(f'df train shape: {df_train.shape}')
print(f'df val shape: {df_val.shape}')

In [ ]:
df_train['verb_class'].value_counts()

In [ ]:
df_val['verb_class'].value_counts()

In [ ]:
df_val.head()

In [ ]:
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')

## weight for classes exporation

In [ ]:
unique_labels = np.array(df_train['verb_class'].unique())
unique_labels.sort()
all_labels = np.array(df_train['verb_class'])

In [ ]:
print(unique_labels)

In [ ]:
all_labels

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=unique_labels, y=all_labels)

In [ ]:
class_weights

In [ ]:
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
class_weights_tensor.dtype

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

Image reshape and save to another dict

In [ ]:
from PIL import Image
import torchvision.transforms as T

In [ ]:
frame = Image.open('frame_0000042685.jpg')
frame.size

In [ ]:
img_path = 'sample_image.jpg'

In [ ]:
img = Image.open(img_path)

In [ ]:
img.size

In [ ]:
resize = T.Resize(size=(100, 100), antialias=True)

In [ ]:
img_resize = resize(img)

In [ ]:
img_resize = img.resize(size=(100, 100))

In [ ]:
img_resize.size

In [ ]:
import os

In [ ]:
root = os.getcwd()
root

In [ ]:
img_resize.save(r"C:\Users\34609\VisualStudio\TFG\sample.jpg")

In [ ]:
dirs = os.listdir(r"C:\Users\34609\VisualStudio\TFG")
dirs

In [ ]:
joined = os.path.join("C:/Users/34609/VisualStudio/TFG/", dirs[4])

In [ ]:
joined

In [ ]:
os.mkdir(r'C:\Users\34609\VisualStudio\TFG\samples')

In [ ]:
if os.path.isdir(r'C:\Users\34609\VisualStudio\TFG'):
    print('Yes')

## Load image with pyvips

In [2]:
import os
vipshome = 'c:\\vips-dev-8.7\\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']

In [3]:
import pyvips

OSError: cannot load library 'libgobject-2.0-0.dll': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0.dll'

In [ ]:
!pip uninstall pyvips

In [ ]:
image = pyvips.Image.new_from_file('some-image.jpg')
a1 = image.numpy()

## Load image with opencv

In [1]:
import cv2

In [53]:
arr = cv2.imread('frame_0000042685.jpg', cv2.IMREAD_UNCHANGED)

In [54]:
arr.shape

(256, 256, 3)

In [55]:
to_tensor = T.ToTensor()
resize = T.Resize(size=(112, 112), antialias=True)

In [85]:
img = to_tensor(arr)

In [86]:
frames = [resize(img).unsqueeze(-1) for i in range(100)]

In [87]:
clip = torch.cat(frames, dim=-1)

In [88]:
clip.size()

torch.Size([3, 112, 112, 100])

In [89]:
pad = (50, 50)
pad

(50, 50)

In [90]:
clip_padded = nn.functional.pad(clip, pad, 'constant', 0)

In [91]:
clip_padded.size()

torch.Size([3, 112, 112, 200])

In [92]:
clip_2 = rearrange(clip_padded, 'c w h t1 -> t1 c w h')
clip_2.size()

torch.Size([200, 3, 112, 112])

In [94]:
clip_3 = clip_2[0:200:2] # take 1 for every 2 frames
clip_3.size()

torch.Size([100, 3, 112, 112])

In [84]:
clip_4 = rearrange(clip_3, 't2 c w h -> c w h t2') 
clip_4.size()

torch.Size([112, 112, 200, 3])

In [43]:
x = torch.randint(low=0, high=0, size=(1,))

RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=0